In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass, field
from enum import Enum
from datetime import datetime, timedelta

In [3]:
def load_and_preprocess_data(filepath: str) -> pd.DataFrame:
    
    act = pd.read_csv("../data/dailyActivity_merged_fin_sum.csv")
   
    
    # 날짜 컬럼을 datetime으로 변환
    act['ActivityDate'] = pd.to_datetime(act['ActivityDate'])
    
    # Mile -> Km 단위 변환
    distance_cols = [col for col in act.columns if 'Distance' in col]
    act[distance_cols] = (act[distance_cols] * 1.60934).round(2)
        
    # Id 컬럼을 문자열로 변환
    act['Id'] = act['Id'].astype(str)
    
    return act
act = load_and_preprocess_data("../data/dailyActivity_merged_fin_sum.csv")
act

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,2016-03-25,11004,11.44,11.44,0.0,4.14,0.74,6.55,0.00,33,12,205,804,1819
1,1503960366,2016-03-26,17609,18.59,18.59,0.0,11.14,1.17,6.29,0.00,89,17,274,588,2154
2,1503960366,2016-03-27,12736,13.73,13.73,0.0,7.50,0.26,5.97,0.00,56,5,268,605,1944
3,1503960366,2016-03-28,13231,14.37,14.37,0.0,5.13,1.27,7.97,0.00,39,20,224,1080,1932
4,1503960366,2016-03-29,12041,12.63,12.63,0.0,3.48,1.75,7.42,0.00,28,28,243,763,1886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1368,8877689391,2016-05-05,14055,17.17,17.17,0.0,8.79,1.32,7.03,0.00,67,15,188,1170,3052
1369,8877689391,2016-05-06,21727,31.12,31.12,0.0,20.58,0.47,9.91,0.00,96,17,232,1095,4015
1370,8877689391,2016-05-07,12332,13.08,13.08,0.0,0.13,1.54,11.25,0.00,105,28,271,1036,4142
1371,8877689391,2016-05-08,10686,13.05,13.05,0.0,1.74,0.32,10.94,0.00,17,4,245,1174,2847


In [4]:
def remove_non_wear_days(df: pd.DataFrame) -> pd.DataFrame:  #미착용 데이터 제거
    non_wear = (df['TotalSteps'] == 0) & \
               (df['TotalDistance'] == 0) & \
               (df['SedentaryMinutes'] >= 1380)
    
    print(f"✓ 미착용일 제거: {non_wear.sum()}개")
    return df[~non_wear].copy()
act = remove_non_wear_days(act)

✓ 미착용일 제거: 124개


In [5]:
def create_derived_features(df: pd.DataFrame) -> pd.DataFrame: #파생 변수 생성
    
    df = df.copy()
    
    # 요일 관련 변수
    df['weekday'] = df['ActivityDate'].dt.day_name()
    df['is_weekend'] = df['weekday'].isin(['Saturday', 'Sunday'])
    
    # 총 활동 시간
    df['TotalActiveMinutes'] = (
        df['VeryActiveMinutes'] + 
        df['FairlyActiveMinutes'] + 
        df['LightlyActiveMinutes']
    )
    
    # 앉아있는 시간 비율
    df['SedentaryRatio'] = df['SedentaryMinutes'] / 1440
    
    # 운동 강도 점수
    df['Intensity_Score'] = (
        (df['VeryActiveMinutes'] * 2) + 
        (df['FairlyActiveMinutes'] * 1.5) + 
        (df['LightlyActiveMinutes'] * 1)
    )
    
    # 효율성 지표 (0으로 나누기 방지)
    df['Efficiency'] = np.where(
        df['TotalActiveMinutes'] > 0,
        df['Intensity_Score'] / df['TotalActiveMinutes'],
        0
    )
    
    # 칼로리 효율 (km당 소모 칼로리)
    df['CaloriesPerKm'] = np.where(
        df['TotalDistance'] > 0,
        df['Calories'] / df['TotalDistance'],
        np.nan
    )
    
    print("✓ 파생 변수 생성 완료")
    return df
act = create_derived_features(act)

✓ 파생 변수 생성 완료


In [43]:
def classify_day_type(df: pd.DataFrame) -> pd.DataFrame:
    """
    DayType 분류
    - Active Day: 걸음 7000+ 또는 활동 60분+
    - Over-Sedentary Day: 앉아있는 비율 75%+
    - Low Engagement Day: 걸음 3000 미만 & 앉아있는 비율 50%+
    - Normal Day: 그 외
    """
    df = df.copy()
    
    conditions = [
        (df['TotalSteps'] >= 7000) | (df['TotalActiveMinutes'] >= 60),
        (df['SedentaryRatio'] >= 0.75) & (df['TotalSteps'] > 0),
        (df['TotalSteps'] < 3000) & (df['SedentaryRatio'] >= 0.50) & (df['TotalSteps'] > 0)
    ]
    
    choices = ['Active Day', 'Over-Sedentary Day', 'Low Engagement Day']
    df['DayType'] = np.select(conditions, choices, default='Normal Day')
    
    print("✓ DayType 분류 완료")
    return df
act = classify_day_type(act)

✓ DayType 분류 완료


In [44]:
def create_calorie_groups(df: pd.DataFrame) -> pd.DataFrame: #칼로리 그룹 분류
    
    df = df.copy()
    
    bins = [0, 1500, 2000, 2500, float('inf')]
    labels = ['1000-1500', '1500-2000', '2000-2500', '2500+']
    df['CalorieGroup'] = pd.cut(df['Calories'], bins=bins, labels=labels, right=False)
    
    print("✓ CalorieGroup 생성 완료")
    print(f"\n칼로리 그룹별 분포:")
    print(df['CalorieGroup'].value_counts().sort_index())
    
    return df

act = create_calorie_groups(act)

✓ CalorieGroup 생성 완료

칼로리 그룹별 분포:
CalorieGroup
1000-1500     61
1500-2000    206
2000-2500    171
2500+        158
Name: count, dtype: int64


In [45]:
def remove_outliers_iqr(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame: # IQR 방법으로 이상치 제거
  
    df_clean = df.copy()
    
    print("\n=== IQR 이상치 제거 ===")
    for col in columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        before_count = len(df_clean)
        df_clean = df_clean[
            (df_clean[col] >= lower_bound) & 
            (df_clean[col] <= upper_bound)
        ]
        after_count = len(df_clean)
        
        if before_count != after_count:
            print(f"{col}: {before_count - after_count}개 제거")
    
    return df_clean
outlier_cols = ['TotalSteps', 'TotalDistance', 'TrackerDistance',
                'LoggedActivitiesDistance', 'VeryActiveDistance',
                'ModeratelyActiveDistance', 'LightActiveDistance',
                'VeryActiveMinutes','FairlyActiveMinutes', 
                'LightlyActiveMinutes', 
                'SedentaryMinutes', 'Calories','CaloriesPerKm']

act = remove_outliers_iqr(act, outlier_cols)


=== IQR 이상치 제거 ===
TotalSteps: 2개 제거
VeryActiveDistance: 52개 제거
ModeratelyActiveDistance: 33개 제거
VeryActiveMinutes: 84개 제거
FairlyActiveMinutes: 31개 제거


In [46]:
def filter_date_range(df: pd.DataFrame, 
                      start_date: Optional[str] = "2016-04-01",
                      end_date: Optional[str] = None) -> pd.DataFrame:   #날짜 범위 필터링 (2016-04-01부터)
    
    start_date = pd.to_datetime(start_date)
        
    if end_date is None:
        end_date = df['ActivityDate'].max()
    else:
        end_date = pd.to_datetime(end_date)
    
    filtered = df[
        (df['ActivityDate'] >= start_date) & 
        (df['ActivityDate'] <= end_date)
    ].copy()
    
    print(f"✓ 날짜 필터링: {start_date.date()} ~ {end_date.date()}")
    print(f"  데이터 행 수: {len(filtered)}")
    
    return filtered

act = filter_date_range(act)


✓ 날짜 필터링: 2016-04-01 ~ 2016-05-12
  데이터 행 수: 394


In [47]:
from typing import Optional
from datetime import datetime, timedelta

class StreakCalculator: #연속 활동일(Streak) 계산 유틸리티
   
    @staticmethod
    def calculate_max_streak(
        dates: pd.Series,
        *,
        start_date: Optional[str] = None,
        end_date: Optional[str] = None
    ) -> int:
        """
        최대 연속 활동일 계산 (필요 시 날짜 범위 제한)

        Args:
            dates: 활동 날짜 시리즈 (예: act.groupby('Id')['ActivityDate'])
            start_date: 포함 시작일 (예: "2016-04-01")
            end_date: 포함 종료일 (None이면 dates의 최대 날짜)

        Returns:
            최대 연속일 (int)
        """
        s = pd.to_datetime(dates, errors="coerce")

        if start_date is not None:
            s = s[s >= pd.to_datetime(start_date)]
        if end_date is not None:
            s = s[s <= pd.to_datetime(end_date)]

        # 날짜만(시간 제거) + 결측 제거 + 중복 제거 + 정렬
        uniq = pd.Series(s.dropna().dt.normalize().unique())
        if uniq.empty:
            return 0

        uniq = uniq.sort_values().to_list()

        max_streak = 1
        current = 1

        for i in range(1, len(uniq)):
            if (uniq[i] - uniq[i - 1]).days == 1:
                current += 1
                if current > max_streak:
                    max_streak = current
            else:
                current = 1

        return max_streak

    @staticmethod
    def calculate_current_streak(
        dates: pd.Series,
        *,
        today: Optional[datetime] = None,
        start_date: Optional[str] = None,
        end_date: Optional[str] = None
    ) -> int:
        """
        '오늘(today)' 기준 현재 진행 중인 연속 활동일 계산
        Args:
            dates: 활동 날짜 시리즈
            today: 기준 날짜(datetime/date). None이면 오늘 날짜 사용.
            start_date: 포함 시작일
            end_date: 포함 종료일

        Returns:
            현재 연속일 (int)
        """
        if today is None:
            today_date = datetime.now().date()
        else:
            # datetime이든 date든 date로 맞추기
            today_date = today.date() if hasattr(today, "date") else today

        s = pd.to_datetime(dates, errors="coerce")

        if start_date is not None:
            s = s[s >= pd.to_datetime(start_date)]
        if end_date is not None:
            s = s[s <= pd.to_datetime(end_date)]

        # date 단위로 변환 + 중복 제거
        ds = sorted(set(s.dropna().dt.date), reverse=True)

        if not ds or ds[0] != today_date:
            return 0

        streak = 1
        for i in range(1, len(ds)):
            expected = today_date - timedelta(days=i)
            if ds[i] == expected:
                streak += 1
            else:
                break

        return streak

streak_df = (
    act.groupby("Id")["ActivityDate"]
       .apply(StreakCalculator.calculate_max_streak)
       .reset_index(name="streak_days")
)
act = act.merge(streak_df, on="Id", how="left")


In [10]:
act.head()

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,...,Calories,weekday,is_weekend,TotalActiveMinutes,SedentaryRatio,Intensity_Score,Efficiency,CaloriesPerKm,CalorieGroup,streak_days
0,1503960366,2016-04-01,12262,12.67,12.67,0.0,5.34,1.34,5.86,0.0,...,1868,Friday,False,268,0.601389,325.5,1.214552,147.434886,1500-2000,21
1,1503960366,2016-04-10,10057,11.23,11.23,0.0,6.44,0.79,3.99,0.0,...,1755,Sunday,True,225,0.511806,275.5,1.224444,156.277827,1500-2000,21
2,1503960366,2016-04-11,10990,11.68,11.68,0.0,3.28,0.92,7.48,0.0,...,1811,Monday,False,256,0.593750,289.0,1.128906,155.051370,1500-2000,21
3,1503960366,2016-04-12,13386,13.90,13.90,0.0,3.03,0.89,9.96,0.0,...,2035,Tuesday,False,375,0.527778,406.5,1.084000,146.402878,2000-2500,21
4,1503960366,2016-04-13,10735,11.22,11.22,0.0,2.53,1.11,7.58,0.0,...,1797,Wednesday,False,257,0.538889,287.5,1.118677,160.160428,1500-2000,21


In [50]:
act.dtypes

Id                                     str
ActivityDate                datetime64[us]
TotalSteps                           int64
TotalDistance                      float64
TrackerDistance                    float64
LoggedActivitiesDistance           float64
VeryActiveDistance                 float64
ModeratelyActiveDistance           float64
LightActiveDistance                float64
SedentaryActiveDistance            float64
VeryActiveMinutes                    int64
FairlyActiveMinutes                  int64
LightlyActiveMinutes                 int64
SedentaryMinutes                     int64
Calories                             int64
weekday                                str
is_weekend                            bool
TotalActiveMinutes                   int64
SedentaryRatio                     float64
Intensity_Score                    float64
Efficiency                         float64
CaloriesPerKm                      float64
CalorieGroup                      category
streak_days

In [51]:
class ActivityLevel(Enum):
    LEVEL_1 = "저활동" #1000-1500
    LEVEL_2 = "보통 활동" #1500-2000
    LEVEL_3 = "활동적" #2000-2500
    LEVEL_4 = "고활동" #2500+
    

In [52]:
class PersistenceType(Enum):
    TYPE_A = "안정적"  # 평균(32일) 이상
    TYPE_B = "유동적"  # 평균(32일) 미만

In [53]:
class PersonaType(Enum):

    """패르소나 유형"""
    Newbie = ("입문자형", ActivityLevel.LEVEL_1, 
                    [PersistenceType.TYPE_A, PersistenceType.TYPE_B ],
                    "막 시작한 사람")
    
    Beginner = ("초보자형", ActivityLevel.LEVEL_2,
                   [PersistenceType.TYPE_B],
                   "운동 수준 낮고 습관화 안 된 사람")
    
    Turtle = ("거북이형", ActivityLevel.LEVEL_2,
                    [PersistenceType.TYPE_A],
                    "습관화는 되었으나 운동강도 낮음")
    
    Burst_Learner = ("벼락치기형", ActivityLevel.LEVEL_3,
                          [PersistenceType.TYPE_B],
                          "중급자이도 운동에 적응이 되었지만, 자주 안함")
    
    Ideal_Student = ("모범생형", ActivityLevel.LEVEL_3,
                      [PersistenceType.TYPE_A],
                      "이상적인 타입. 운동강도 좋고 횟수도 좋음")
    
    Lazy_Genius = ("게으른 천재형", ActivityLevel.LEVEL_4,
                      [PersistenceType.TYPE_B],
                      "운동에 어느정도 수준이 있지만 횟수 적음")
    
    Veteran = ("고인물형", ActivityLevel.LEVEL_4,
                      [PersistenceType.TYPE_A],
                      "운동강도 높고 빈도수도 좋음. 부상위험도 높으니 관리 필요")
    



    @property
    def name(self) -> str:
        return self.value[0]
    
    @property
    def activity_level(self) -> ActivityLevel:
        return self.value[1]
    
    @property
    def persistence_types(self) -> List[PersistenceType]:
        return self.value[2]
    
    @property
    def description(self) -> str:
        return self.value[3]

In [54]:
@dataclass
class UserMetrics:
    """사용자 월간 활동 지표"""
    user_id: str
    avg_calories: float
    avg_steps: float
    avg_distance: float
    avg_efficiency: float
    
    # 활동 시간 (분)
    avg_very_active_min: float
    avg_fairly_active_min: float
    avg_lightly_active_min: float
    avg_total_active_min: float
    
    # 지속성 지표
    total_days: int
    active_days: int  # TotalSteps > 0인 날
    max_streak: int   # 최대 연속 활동일
    active_ratio: float  # active_days / total_days
    
    # DayType 비율
    over_sedentary_ratio: float
    low_engagement_ratio: float
    active_day_ratio: float
    
    # 칼로리 그룹 (최빈값)
    dominant_calorie_group: str
    
    def __post_init__(self):
        """활동 비율 계산"""
        self.active_ratio = self.active_days / self.total_days if self.total_days > 0 else 0


In [55]:
@dataclass
class PersonaInfo: #페르소나 분류 결과
    
    user_id: str
    persona: PersonaType
    activity_level: ActivityLevel
    persistence_type: PersistenceType
    
    # 상세 지표
    metrics: UserMetrics
    
    # 목표 및 추천
    short_term_goal: str = ""
    medium_term_goal: str = ""
    long_term_goal: str = ""
    recommended_programs: List[str] = field(default_factory=list)
    
    def get_summary(self) -> str: #페르소나 요약 정보
        
        return f"""
        {'='*50}
        사용자 ID: {self.user_id}
        페르소나: {self.persona.name_kr}
        활동 등급: {self.activity_level.value}
        지속성: {self.persistence_type.value}

        [주요 지표]
        - 평균 칼로리: {self.metrics.avg_calories:.0f} kcal
        - 평균 걸음수: {self.metrics.avg_steps:.0f} 보
        - 활동 일수: {self.metrics.active_days}/{self.metrics.total_days}일
        - 효율성: {self.metrics.avg_efficiency:.2f}

        [특징]
        {self.persona.description}

        [단기 목표 (1주)]
        {self.short_term_goal}

        [중기 목표 (1개월)]
        {self.medium_term_goal}

        [장기 목표 (3개월)]
        {self.long_term_goal}

        [추천 프로그램]
        {"\n".join(f'• {prog}' for prog in self.recommended_programs)}
        {'='*50}
        """


In [73]:
class PersonaClassifier:
    """페르소나 자동 분류 시스템"""
    
    def __init__(self):
        self.persona_rules = self._initialize_rules()
    
    def _initialize_rules(self) -> Dict: # """페르소나 분류 규칙 정의 (2차원 매트릭스)"""
   
        return {
        
            PersonaType.Newbie: {
                'CalorieGroup': "1000 - 1500",
                'activity_days_min': 32,  # 안정적
                'description': '입문자 (안정적&유동적, 저활동)'
            },
            PersonaType.Beginner: {
            'CalorieGroup': "1000 - 1500",
            'activity_days_max': 31,  # 유동적
            'description': '입문자(유동) (유동적, 보통 활동)'
            },
        
            PersonaType.Turtle: {
            'CalorieGroup': "1500 - 2000",
            'activity_days_min': 32,  # 안정적
            'description': '거북이형 (안정적, 보통활동)'
            },
            PersonaType.Burst_Learner: {
            'CalorieGroup': "2000 - 2500",
            'activity_days_max': 31,  # 유동적
            'description': '벼락치기형 (유동적, 활동적)'
            },
        
       
            PersonaType.Ideal_Student: {
            'CalorieGroup': "2000 - 2500",
            'activity_days_min': 32,  # 안정적
            'description': '모범생형 (안정적, 활동적)'
             },
            
            PersonaType.Lazy_Genius: {
            'CalorieGroup': "2500+",
            'activity_days_max': 31,  # 유동적
            'description': '게으른 천재형 (유동적, 고활동)'
            },
        
       
            PersonaType.Veteran: {
            'CalorieGroup': "2500+",
            'activity_days_min': 32,  # 안정적
            'description': '고인물형 (안정적, 고활동)'
            },
        }
    
    def calculate_user_metrics(self, df: pd.DataFrame, user_id: str) -> UserMetrics:
        """사용자별 월간 지표 계산"""
        user_data = df[df['Id'] == user_id].copy()
        
        if len(user_data) == 0:
            raise ValueError(f"User {user_id}의 데이터가 없습니다.")
        
        # 연속 활동일 계산
        user_data = user_data.sort_values('ActivityDate')
        user_data['is_active'] = user_data['TotalSteps'] > 0
        
        max_streak = self._calculate_max_streak(user_data['is_active'].values)
        
        # DayType 비율
        day_type_counts = user_data['DayType'].value_counts(normalize=True)
        
        return UserMetrics(
            user_id=user_id,
            avg_calories=user_data['Calories'].mean(),
            avg_steps=user_data['TotalSteps'].mean(),
            avg_distance=user_data['TotalDistance'].mean(),
            avg_efficiency=user_data['Efficiency'].mean(),
            
            avg_very_active_min=user_data['VeryActiveMinutes'].mean(),
            avg_fairly_active_min=user_data['FairlyActiveMinutes'].mean(),
            avg_lightly_active_min=user_data['LightlyActiveMinutes'].mean(),
            avg_total_active_min=user_data['TotalActiveMinutes'].mean(),
            
            total_days=len(user_data),
            active_days=(user_data['TotalSteps'] > 0).sum(),
            max_streak=max_streak,
            active_ratio=(user_data['TotalSteps'] > 0).mean(),
            
            over_sedentary_ratio=day_type_counts.get('Over-Sedentary Day', 0),
            low_engagement_ratio=day_type_counts.get('Low Engagement Day', 0),
            active_day_ratio=day_type_counts.get('Active Day', 0),
            
            dominant_calorie_group=user_data['CalorieGroup'].mode()[0] if len(user_data) > 0 else 'Unknown'
        )
    
    def _calculate_max_streak(self, is_active_array: np.ndarray) -> int:
        """최대 연속 활동일 계산"""
        if len(is_active_array) == 0:
            return 0
        
        max_streak = 0
        current_streak = 0
        
        for is_active in is_active_array:
            if is_active:
                current_streak += 1
                max_streak = max(max_streak, current_streak)
            else:
                current_streak = 0
        
        return max_streak
    
    def classify_activity_level(self, calorie_group: str) -> ActivityLevel:  
        """
    CalorieGroup 문자열을 ActivityLevel로 매핑
    
    Args:
        calorie_group: '1000-1500', '1500-2000', '2000-2500', '2500+' 중 하나
    
    Returns:
        ActivityLevel Enum
    """
        mapping = {
        '1000-1500': ActivityLevel.LEVEL_1,  # 저활동
        '1500-2000': ActivityLevel.LEVEL_2,  # 보통활동
        '2000-2500': ActivityLevel.LEVEL_3,  # 활동적
        '2500+': ActivityLevel.LEVEL_4       # 고활동
        }
    
        return mapping.get(calorie_group, ActivityLevel.LEVEL_1)
    
    def classify_persistence(self, metrics: UserMetrics) -> PersistenceType:
        if metrics.active_days >= 32:
            return PersistenceType.TYPE_A  # 안정적
        else:
            return PersistenceType.TYPE_B  # 유동적
    
    def classify_persona(self, metrics: UserMetrics) -> PersonaType:
    
    # 활동 등급 및 지속성 판단
        activity_level = self.classify_activity_level(metrics.dominant_calorie_group)
        persistence = self.classify_persistence(metrics)
    
    # LEVEL_4 (2500+ kcal)
        if activity_level == ActivityLevel.LEVEL_4:
            if persistence == PersistenceType.TYPE_A:
                return PersonaType.Veteran  # 고인물형
            else:  # TYPE_B
                return PersonaType.Lazy_Genius  # 게으른 천재형
    
    # LEVEL_3 (2000-2500 kcal)
        elif activity_level == ActivityLevel.LEVEL_3:
            if persistence == PersistenceType.TYPE_A:
                return PersonaType.Ideal_Student  # 모범생형
            else:  # TYPE_B
                return PersonaType.Burst_Learner  # 벼락치기형
    
    # LEVEL_2 (1500-2000 kcal)
        elif activity_level == ActivityLevel.LEVEL_2:
            if persistence == PersistenceType.TYPE_A:
                return PersonaType.Turtle  # 거북이형
            else:  # TYPE_B
                return PersonaType.Beginner  # 초보자형
    
    # LEVEL_1 (1000-1500 kcal) - 기본값
        else:
            return PersonaType.Newbie  # 입문자형
    
    def generate_goals_and_programs(self, persona: PersonaType, 
                               metrics: UserMetrics) -> Tuple[str, str, str, List[str]]:
        """페르소나별 목표 및 프로그램 생성"""
    
        goals_map = {
            PersonaType.Newbie: (
                "하루 3,000보 달성 주 3회",
                "1500 kcal 그룹 진입",
                "Beginner로 성장",
                ["3분 스트레칭 챌린지", "출퇴근길 걷기", "기초 운동 루틴"]
            ),
            PersonaType.Beginner: (
                "주 5회 운동 습관 만들기",
                "7일 연속 활동 달성",
                "Turtle로 성장 (습관화)",
                ["매일 아침 운동 알림", "운동 일지 작성", "친구와 함께 운동"]
            ),
            PersonaType.Turtle: (
                "운동 강도 높이기 (주 2회)",
                "2000 kcal 그룹 진입",
                "Ideal_student로 성장",
                ["인터벌 트레이닝", "근력 운동 추가", "속도 챌린지"]
            ),
            PersonaType.Burst_Learner: (
                "주 5회 이상 운동",
                "연속 7일 달성",
                "Ideal_student로 성장",
                ["스트릭 유지 보상", "운동 루틴 자동화", "습관 트래커"]
            ),
            PersonaType.Ideal_Student: (
                "현재 수준 유지",
                "고강도 운동 도전 (월 2회)",
                "Veteran 도전 or 멘토 활동",
                ["다양한 운동 시도", "커뮤니티 리더", "신규 유저 멘토링"]
            ),
            PersonaType.Lazy_Genius: (
                "운동 빈도 늘리기 (주 4회→5회)",
                "연속 7일 달성",
                "Veteran으로 성장",
                ["운동 스케줄링", "알림 강화", "습관화 프로그램"]
            ),
            PersonaType.Veteran: (
                "부상 예방 관리",
                "휴식 및 회복 최적화",
                "운동 다양화 및 전문화",
                ["스마트 회복 프로그램", "부상 방지 스트레칭", "전문 코칭"]
            )
        }
    
        return goals_map.get(persona, ("", "", "", []))
    
    def classify_user(self, df: pd.DataFrame, user_id: str) -> PersonaInfo:
        """사용자 종합 분류"""
        # 1. 지표 계산
        metrics = self.calculate_user_metrics(df, user_id)
        
        # 2. 활동 등급 분류
        activity_level = self.classify_activity_level(metrics.avg_calories)
        
        # 3. 지속성 분류
        persistence = self.classify_persistence(metrics)
        
        # 4. 페르소나 분류
        persona = self.classify_persona(metrics)
        
        # 5. 목표 및 프로그램 생성
        short, medium, long, programs = self.generate_goals_and_programs(persona, metrics)
        
        return PersonaInfo(
            user_id=user_id,
            persona=persona,
            activity_level=activity_level,
            persistence_type=persistence,
            metrics=metrics,
            short_term_goal=short,
            medium_term_goal=medium,
            long_term_goal=long,
            recommended_programs=programs
        )
    
    def classify_all_users(self, df: pd.DataFrame) -> Dict[str, PersonaInfo]:
        """전체 사용자 분류"""
        results = {}
        unique_users = df['Id'].unique()
        
        print(f"\n=== 전체 사용자 페르소나 분류 시작 ===")
        print(f"총 사용자 수: {len(unique_users)}")
        
        for user_id in unique_users:
            try:
                results[user_id] = self.classify_user(df, user_id)
                print(f"✓ {user_id}: {results[user_id].persona.name_kr}")
            except Exception as e:
                print(f"✗ {user_id}: 분류 실패 - {str(e)}")
        
        return results

In [74]:
def analyze_single_user(df: pd.DataFrame, user_id: str):
    """
    사용자 ID 입력 → 자동으로 페르소나 분석 + 상세 리포트 출력
    """
    # 1. PersonaClassifier 사용해서 자동 분류
    classifier = PersonaClassifier()
    persona_info = classifier.classify_user(df, user_id)
    
    # 2. 예쁘게 출력
    print(f"사용자: {user_id}")
    print(f"페르소나: {persona_info.persona.value[0]}")
    print(f"목표: {persona_info.short_term_goal}")

In [75]:
# 원하는 사용자 ID 입력
analyze_single_user(act, "1624580081")

사용자: 1624580081
페르소나: 입문자형
목표: 하루 3,000보 달성 주 3회
